## 01. standardize countries/states classification

In [125]:
import pandas as pd 
from os.path import join 
from os import getcwd
from datetime import datetime
import statistics as stats
import numpy as np
import requests
import io
from utils.DBReader import read_table
from datetime import datetime 

def save_fig(fig,saved_name):
    fig.savefig(saved_name)
    print("Figure {} has been successfully saved.".format(saved_name))

def split(item): 
    return item.split('T')[0]
    
def hashrate_filter(country_name,manual_adjust_country_list):
    """
    This function takes filters for country
    that will be treated as world general
    
    Input: 
    country_name = str 
    
    Output: 
    Boolean
    """
    
    if country_name in manual_adjust_country_list:
        return 'World'
    else:
        return country_name

In [126]:
folder = 'Data/'


df2_url = "https://ccaf.io/cbeci/api/v1.2.0/eth/pos/download/network_power_demand" #CBECI Network Ethereum Power Demand Estimate

df3_path = folder+'share-elec-by-source.csv'#Share of Electricity by source from our world in data

country_map_path = folder+'country_map_file.xlsx' #Country name standardize file

#Accumulative Ethereum Electricity Consumption Estimate - CBECI 
eleCum_url = 'https://ccaf.io/cbeci/api/v1.2.0/eth/pos/download/total_electricity_consumption/monthly'

In [127]:
## 1. standardize country classification
country_classification = pd.read_excel(folder+'udpated-Country classifications.xlsx')
country_classification.drop('Region',axis = 1,inplace = True)
country_classification = country_classification.loc[1:,]
country_classification_long = pd.melt(
    country_classification,
    id_vars=None,
    value_vars=None,
    var_name='region',
    value_name='country/state'
    
)
country_classification_long = country_classification_long.dropna()
country_classification_long['country/state']= country_classification_long['country/state'].str.strip()
country_classification_long['level']='countries'
state_level = ['us','us-west','us-east','china']
filt = [x in state_level for x in country_classification_long['region']]
country_classification_long.loc[filt,'level'] = 'states'

In [128]:
country_id = read_table(db_connection='custom_data',table='countries',schemas = None, all_col = True)
country_id = country_id[['country','code','id','code3']]
country_id.rename({'id':'country_id'},axis = 1,inplace = True)
province_id = read_table(db_connection='custom_data',table='mining_map_provinces',schemas = None, all_col = True)
province_id =province_id[['name','country_id','code']].drop_duplicates()

In [129]:
country_id

,country,code,country_id,code3
0,Cabo Verde,CV,40,CPV
1,China,CN,129,CHN
2,Belgium,BE,32,BEL
3,Andorra,AD,22,AND
4,Colombia,CO,42,COL
...,...,...,...,...
214,Kosovo,KV,216,XKX
215,Lebanon,LB,190,LBN
216,Lithuania,LT,192,LTU
217,British Virgin Islands,BV,215,BVT


In [130]:
province_id = pd.merge(province_id,country_id,how ='left',on = 'country_id')
province_id.drop(['code_y'],axis = 1,inplace = True)
province_id.rename({'code_x':'province_code'},axis = 1,inplace = True)

In [131]:
countries_classification = country_classification_long[country_classification_long['level'] == 'countries']
country_classification_df = pd.merge(countries_classification,country_id,how = 'left',left_on = 'country/state',right_on = 'country')
country_classification_df = country_classification_df.drop(['country','code','country_id'],axis = 1)

In [132]:
country_dict = {'Russia':'RUS', 
                'Czech Republic (Czechia)':'CZE', 
                'Slovakia':'SVK',
               'Holy See (Vatican City)':'OWID_WRL', 
                'Iran':'IRN', 
                'South Korea':'KOR', 
                'Yemen':'YEM',
               'North Korea':'PRK', 
                'Syria':'SYR', 
                'Laos':'LAO', 
                'Kyrgyzstan':'KGZ', 
                'State of Palestine':'PSE',
                'Brunei':'BRN', 
                'Taiwan (China)':'TWN', 
                'Hong Kong (China)':'HKG', 
                'Macao (China)':'MAC',
                'Taiwan':'TWN', 
                'Hong Kong':'HKG', 
                'Republic of Moldova':'MDA',
                'Saint Pierre and Miquelon':'SPM',
                'Macao':'MAC', 
                'Others':'OWID_WRL'}

In [133]:
for index, row in country_classification_df.iterrows():
    if row['country/state'] in country_dict.keys():
        country_classification_df.at[index,'code3'] = country_dict[row['country/state']]


In [134]:
country_classification_df = pd.merge(country_classification_df,country_id,how = 'left',on = 'code3')
country_classification_df = country_classification_df.drop('code',axis = 1)

In [135]:
country_classification_df

,region,country/state,level,code3,country,country_id
0,europe,Russia,countries,RUS,Russian Federation,181.0
1,europe,Germany,countries,DEU,Germany,144.0
2,europe,United Kingdom,countries,GBR,United Kingdom,212.0
3,europe,France,countries,FRA,France,56.0
4,europe,Italy,countries,ITA,Italy,67.0
...,...,...,...,...,...,...
142,taiwan,Taiwan (China),countries,TWN,"Taiwan, China",100.0
143,ukraine,Ukraine,countries,UKR,Ukraine,207.0
144,japan,Japan,countries,JPN,Japan,151.0
145,sweden,Sweden,countries,SWE,Sweden,208.0


In [136]:
states_classification = country_classification_long[country_classification_long['level'] == 'states']

In [137]:
states_classification_df = pd.merge(states_classification,province_id,how = 'left',left_on = 'country/state',right_on = 'name')

In [138]:
filt = pd.isnull(states_classification_df['name'])
states_classification_df.loc[filt]

,region,country/state,level,name,country_id,province_code,country,code3
93,china,Other,states,NaN,NaN,NaN,NaN,NaN


In [139]:
states_classification_df = states_classification_df.drop(['name'],axis = 1)

In [140]:
classification_df = country_classification_df.append(states_classification_df)

C:\Users\clair\AppData\Local\Temp\ipykernel_109440\2264040123.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  classification_df = country_classification_df.append(states_classification_df)


In [141]:
classification_df.to_excel(folder+'updated_country_classification_clean.xlsx',index = False)

### add more state level data for US

In [65]:
us = pd.read_excel(folder+"electricity-prod-source-stacked-US-state.xlsx")
additional_province = pd.read_excel(folder+'annual_generation_state.xlsx',header =[1])

In [66]:
filt = (additional_province['TYPE OF PRODUCER']=='Total Electric Power Industry')
tempt1 = additional_province.loc[filt,]

## correct categories
tempt1 = pd.pivot(tempt1,index =['YEAR','STATE','TYPE OF PRODUCER'], columns = 'ENERGY SOURCE', 
         values = 'GENERATION (Megawatthours)')
tempt1 = tempt1.fillna(0)
tempt1 = tempt1.reset_index(drop = False)
tempt1['Gas'] = tempt1['Natural Gas'] + tempt1['Other Gases']
tempt1['Hydroelectric'] = tempt1['Hydroelectric Conventional'] +tempt1['Pumped Storage']
tempt1['Other'] = tempt1['Geothermal'] + tempt1['Wood and Wood Derived Fuels'] + tempt1['Other Biomass']
tempt1.rename({'Petroleum':'Oil'},axis = 1,inplace = True)
reserve_col = ['YEAR', 'STATE', 'TYPE OF PRODUCER', 'Coal','Hydroelectric',
              'Gas','Nuclear','Other','Oil','Solar Thermal and Photovoltaic','Wind','Total']
tempt1 = tempt1[reserve_col]

tempt1 = pd.melt(tempt1, id_vars = ['YEAR','STATE','TYPE OF PRODUCER'],value_name = 'GENERATION (Megawatthours)').sort_values(['YEAR','STATE','ENERGY SOURCE']).reset_index(drop = True)
tempt1 = tempt1[tempt1['YEAR']>=2004].reset_index(drop = True)

In [67]:
## add percentage share
tempt1['Total'] =tempt1.groupby(['YEAR','STATE'])['GENERATION (Megawatthours)'].transform('sum')/2
tempt1['ENERGY SHARE'] = tempt1['GENERATION (Megawatthours)']/tempt1['Total']
tempt1.drop('Total',axis = 1,inplace = True)

In [68]:
tempt1.to_excel(folder+"updated-electricity-prod-source-stacked-US-state.xlsx",index = False)

### prepare updated miner-address-region

In [142]:
## merge the address to updated-miner-to-region file to produce updated-ccaf-miner-address-region.xlsx
miner_address_path = folder+'ccaf-miner-address-region.xlsx'
miner_path = folder+'updated-miner-to-region-mix.xlsx'

miner_address = pd.read_excel(miner_address_path)
miner = pd.read_excel(miner_path)

In [143]:
### check if miner in miner address
new_miner = miner['miner'].unique()
print("The following miners not in miner address book")
print([x for x in new_miner if x not in miner_address['miner'].unique()])

The following miners not in miner address book
[]


In [144]:
### 1. split miner address book to keep and updated - (updated all)
updated_filt = [x in new_miner for x in miner_address['miner']]
miner_address.loc[updated_filt]

,address,miner,description,europe,us,asia,china,russia,unknown,url,matched
0,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,2Miners,2Miners: PPLNS,95.0,4.0,1.0,NaN,NaN,NaN,https://solo-eth.2miners.com,1.0
1,0x002e08000acbbae2155fab7ac01929564949070d,2Miners,2Miners: SOLO,95.0,4.0,1.0,NaN,NaN,NaN,https://solo-eth.2miners.com,1.0
2,0xc839ee5542b4e8413246b3634c5c739fea949562,AlphaPool,AlphaPool,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
3,0x45a36a8e118c37e4c47ef4ab827a7c9e579e11e2,AntPool,AntPool,NaN,NaN,NaN,1.0,NaN,NaN,https://antpool.com,1.0
4,0xa855c20a1351acd2690c716e2709c7dff3978d12,AntPool,AntPool,NaN,NaN,NaN,1.0,NaN,NaN,https://antpool.com,1.0
...,...,...,...,...,...,...,...,...,...,...,...
96,0x5c23e54fe46ef9181e4403d6e1dbb9aa21c0b185,hloongpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,0x2a98776c7e13ed1c240858bd241dcf95fc1928b4,myminers.org,myminers.org: Solo,1.0,NaN,NaN,NaN,NaN,NaN,https://solo-eth.myminers.org,1.0
98,0x005e288d713a5fb3d7c9cf1b43810a98688c7223,xnpool.cn,xnpool.cn,32.0,14.0,54.0,NaN,NaN,NaN,https://www.xnpool.com/,1.0
99,0xe4bdced60430a90f31dba03524dd5d15a2670649,xnpool.cn,xnpool.cn,32.0,14.0,54.0,NaN,NaN,NaN,https://www.xnpool.com/,1.0


In [145]:
miner_address_tomerge = miner_address[['address', 'miner', 'description', 'url']]

In [146]:
miner['matched'] =1

In [147]:
output = pd.merge(miner_address_tomerge, miner, how = 'left', on ='miner')

In [150]:
reorder = ['address', 'miner', 'description', 'europe', 'us', 'europe-west',
       'europe-east', 'north-america', 'asia-east', 'asia', 'china',
       'russia', 'unknown', 'url','matched']

In [151]:
output = output[reorder]

In [152]:
output.to_excel(folder+'updated-ccaf-miner-address-region.xlsx',index= False)